# Search the C-STAC API by bounding box or geojson
Items in the API may be searched by a bounding box or GeoJSON. Items intersecting the bounding box or GeoJSON are returned.

## Import dependencies

In [30]:
from typing import List, Dict, Optional
import urllib.parse
import json
import httpx
import logging


logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

## Base URL for the API
For API documentation, see https://c-stac-api.c-core.app/docs
C-STAC-API implements the STAC API specification. For further details of the API search specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/

In [31]:
api_url = "https://c-stac-api.c-core.app"

## List all collection IDs in the catalog

In [48]:
def list_collection_ids(url: str = api_url) -> List[str]:
    """List collection IDs in the catalog."""
    get_url = f"{url}/collections"
    
    with httpx.Client(timeout = 20) as client:
        response = client.get(get_url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    collections = response_body["collections"]
    collection_ids = [collection["id"] for collection in collections]
    
    return collection_ids
    

In [49]:
collection_ids = list_collection_ids()
collection_ids

['joplin',
 'ice-drift-feature-tracking',
 'floe-edge-convergence',
 'floe-edge-coherence',
 'floe-edge-polygons',
 'floe-edge-linestrings']

## Collection names
The database names differ from the common names assigned to the products for reasons of development history. Here is a map:

- `joplin`: A test collection that is unrelated to the sea ice products.
- `ice-drift-feature-tracking`: Ice tracking.
- `floe-edge-convergence`: Land-fast ice motion.
- `floe-edge-coherence`: Tidal cracks.
- `floe-edge-polygons`: Floe edge as a polygon.
- `floe-edge-linestrings`: Floe edge as a linestring.

## List items in collection
This example uses the GET method, which may be easier for simple listings, compared to the POST method. A POST example appears later.

In [50]:
def list_items( 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""

    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

Note we can sort by `id`. For descending sort, specify `-id`.

In [51]:
items = list_items("floe-edge-linestrings", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&limit=2&sortby=-id


['2021-12-28T12:34:53+00:00', '2021-12-24T13:07:39+00:00']

For ascending sort, specify `+id`.

In [52]:
items = list_items("floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&limit=2&sortby=%2Bid


['2019-12-16T12:25:42+00:00', '2019-12-18T12:25:42+00:00']

## Filter by location with bbox
This is an example of search by the GET method, which is a convenient way to search by a bounding box (bbox). A search by POST method for more complex queires is demonstarted later.

For further API details, see https://c-stac-api.c-core.app/docs#/default/Search_search_get .
For the API specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/getItemSearch .

The GET method accepts a `bbox` parameter. Here, we specify rough bounding boxes for Resolute, Pond Inlet, and the world. The bbox order is `[lower left longitude, lower left latitude, upper right longitude, upper right latitude]`.

In [53]:
bboxes = {
    "resolute": [-97.1, 74.3, -92.5, 75.0],
    "pond_inlet": [-80.6, 72.2, -75.1, 73.2],
    "world": [-180, -90, 180, 90],
}

A convenience function for requesting a search GET method with a bbox.

In [54]:
def list_items_in_bbox(
    bbox: List[float], 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    bbox_parameter = ",".join([str(coordinate) for coordinate in bbox])
    
    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&bbox={bbox_parameter}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)

    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

In [55]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-id


['2021-12-24T11:30:21+00:00', '2021-12-19T12:10:31+00:00']

In [56]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bid


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

We can also sort by `datetime`. Because `id` includes the value of the datetime, sorting by either probably results in the same results.

In [57]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-datetime


['2021-12-24T11:30:21+00:00', '2021-12-19T12:10:31+00:00']

In [58]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bdatetime


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

## Filter by location with geojson
This is an example of search by the POST method. It requires providing a JSON body with the search request, but allows for more complex queries, such as intersection with a GeoJSON. 

For further API details, see https://c-stac-api.c-core.app/docs#/default/Search_search_post .
For the API specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/postItemSearch

Here, we specify GeoJSON polygons surrounding Resolute, Pond Inlet, and the world.

In [79]:
geojsons = {
    "resolute": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -97.09716796875,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              75.00636121985819
            ],
            [
              -97.09716796875,
              75.00636121985819
            ],
            [
              -97.09716796875,
              74.27612190544454
            ]
          ]
        ]
    },
    "pond_inlet": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.606689453125,
              72.18852591070342
            ],
            [
              -75.1025390625,
              72.18852591070342
            ],
            [
              -75.1025390625,
              73.18543401519665
            ],
            [
              -80.606689453125,
              73.18543401519665
            ],
            [
              -80.606689453125,
              72.18852591070342
            ]
          ]
        ]
    },
    "world": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -180,
              -90
            ],
            [
              180,
              -90
            ],
            [
              180,
              90
            ],
            [
              -180,
              90
            ],
            [
              -180,
              -90
            ]
          ]
        ]
    }
}

A convenience function for requesting a search POST method with an intersecting GeoJSON.

In [67]:
def list_items_in_geojson(
    geojson: Dict, 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[List[Dict]] = None,
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    post_url = f"{url}/search"
    
    data = {
        "intersects": geojson,
        "collections": [collection_id],
        "limit": limit,
        "sortby": sortby,
    }
    
    with httpx.Client(timeout = 20) as client:
        log.info(post_url)
        log.info(data)
        response = client.post(post_url, data=json.dumps(data))
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

When searching with the POST method, `sortby` accespts an array of dictionaries specifying the `field` and `direction`. Here is a datetime descending sort example.

In [70]:
sortby = [{
    "field": "datetime",
    "direction": "desc",
}]

items = list_items_in_geojson(geojsons["pond_inlet"], "floe-edge-linestrings", sortby=sortby)
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-80.606689453125, 72.18852591070342], [-75.1025390625, 72.18852591070342], [-75.1025390625, 73.18543401519665], [-80.606689453125, 73.18543401519665], [-80.606689453125, 72.18852591070342]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 2, 'sortby': [{'field': 'datetime', 'direction': 'desc'}]}


['2022-03-24T12:18:28+00:00', '2022-03-21T11:53:57+00:00']

Here is a datetime ascending sourt example.

In [72]:
sortby = [{
    "field": "datetime",
    "direction": "asc",
}]

items = list_items_in_geojson(geojsons["pond_inlet"], "floe-edge-linestrings", sortby=sortby)
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-80.606689453125, 72.18852591070342], [-75.1025390625, 72.18852591070342], [-75.1025390625, 73.18543401519665], [-80.606689453125, 73.18543401519665], [-80.606689453125, 72.18852591070342]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 2, 'sortby': [{'field': 'datetime', 'direction': 'asc'}]}


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

## Example
Let's plot the most recent tidal crack data near Resolute.

In [80]:
sortby = [{
    "field": "datetime",
    "direction": "desc",
}]

items = list_items_in_geojson(geojsons["resolute"], "floe-edge-linestrings", sortby=sortby, limit=1)
item = items["features"][0]
item

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-97.09716796875, 74.27612190544454], [-92.5103759765625, 74.27612190544454], [-92.5103759765625, 75.00636121985819], [-97.09716796875, 75.00636121985819], [-97.09716796875, 74.27612190544454]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 1, 'sortby': [{'field': 'datetime', 'direction': 'desc'}]}


{'type': 'Feature',
 'geometry': {'coordinates': [[[-80.767476, 74.415217],
    [-80.767476, 78.146234],
    [-98.151094, 78.146234],
    [-98.151094, 74.415217],
    [-80.767476, 74.415217]]],
  'type': 'Polygon'},
 'properties': {'datetime': '2021-12-24T13:07:39+00:00'},
 'id': 'floe-edge-linestrings-2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z',
 'bbox': [-98.151094, 74.415217, -80.767476, 78.146234],
 'assets': {'geojson': {'title': 'GeoJSON',
   'description': 'Vector data in GeoJSON format',
   'start_datetime': None,
   'end_datetime': None,
   'created': None,
   'updated': None,
   'platform': None,
   'instruments': None,
   'constellation': None,
   'mission': None,
   'providers': None,
   'gsd': None,
   'href': 'https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-linestrings/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z.geojson',
   'type': 'application/geo+json',
   'roles': ['vector-data']},
  'vect